In [210]:
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor

model_id = 'hongseongpil/Wav2Vec2.0_zeroth_Ko'

model = Wav2Vec2ForCTC.from_pretrained(model_id,output_attentions=True)
processor = Wav2Vec2Processor.from_pretrained(model_id)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [222]:
from Tokenize_Kor import decompose_tokens
import torch
from pydub.effects import normalize
from pydub import AudioSegment 
from pydub.silence import detect_nonsilent
import numpy as np
def getAudioInput(source_path):
    try:
        audio = AudioSegment.from_file(source_path).set_frame_rate(16000).set_sample_width(2).set_channels(1)
        if(audio.duration_seconds > 10):
            audio = audio[0:1000*10]
        result = detect_nonsilent(audio,min_silence_len=200,silence_thresh=-60)
        newaudio = AudioSegment.empty()
        for index in result:
            newaudio += audio[index[0]:index[1]]
        newaudio = normalize(newaudio)
        return newaudio
    except Exception as e:
        print("오류 발생:", e)
def PrintAudioInfo(audio):
    channels = audio.channels
    sample_rate = audio.frame_rate
    print("Channels:", channels)
    print("Sample rate:", sample_rate)
    print("Duration: ", audio.duration_seconds)
    print("Bit depth:", audio.sample_width, "bits") 
    print("len samples:", len(np.array(audio.get_array_of_samples())))
    display(audio)

In [231]:
audio_input = getAudioInput("test1.wav")
model.eval()
input = processor(np.array(audio_input.get_array_of_samples(),dtype=np.float32), sampling_rate=16000, return_tensors="pt").input_values[0]
with torch.no_grad():
    input_values = torch.tensor(input).unsqueeze(0)
    logits = model(input_values).logits


C:\Users\zx288\AppData\Local\Temp\ipykernel_16852\1999888468.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input).unsqueeze(0)


In [234]:
import torch

sorted_tensor, indices = torch.sort(logits, dim=2, descending=True)

outputs = list()
for i,v in enumerate(indices[0]):
    outputs.append(processor.tokenizer.convert_ids_to_tokens(v[0].item()))
for i,v in enumerate(outputs):
    print(sorted_tensor[0][i][:3],v)

tensor([12.5506,  4.3925,  3.6470]) ㅇ
tensor([11.7529,  5.2504,  4.4889]) ㅡ
tensor([15.0512,  0.9118,  0.3162]) [PAD]
tensor([12.6545,  4.7039,  3.7945]) ㅁ
tensor([14.2822,  2.3603,  0.6484]) [PAD]
tensor([14.1934,  0.6531, -0.2064]) [PAD]
tensor([13.0712,  1.8187,  0.2361]) [PAD]
tensor([13.2313,  1.1596,  0.3507]) [PAD]
tensor([13.5008,  0.6169,  0.2383]) [PAD]
tensor([13.8909,  0.4174, -0.5245]) [PAD]
tensor([13.9278, -0.2196, -0.9259]) [PAD]
tensor([14.4523, -0.5629, -0.7464]) [PAD]
tensor([14.9401,  0.6581,  0.3584]) [PAD]
tensor([14.6854,  0.8815,  0.5193]) [PAD]
tensor([13.8892,  3.0316,  2.2010]) ㅅ
tensor([12.9898,  4.7764,  2.1064]) ㅅ
tensor([12.4195,  1.9726,  1.9559]) ㅓ
tensor([14.8119,  2.5387,  0.4654]) [PAD]
tensor([14.9663,  3.6826, -0.6118]) [PAD]
tensor([14.8245,  3.8650, -1.0412]) [PAD]
tensor([14.1845,  5.5710, -0.9412]) [PAD]
tensor([13.2673,  3.2142,  2.5330]) ㅇ
tensor([10.6634,  8.9831,  0.3749]) [PAD]
tensor([14.4379,  1.6508, -0.4993]) [PAD]
tensor([12.2520,  6.

In [233]:

predlogits = torch.argmax(logits, dim=-1)[0]


outputs = processor.decode(predlogits,output_char_offsets=True)
time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate
PrintAudioInfo(audio_input)
for w in outputs['char_offsets']:
    start_offset = round (w[ "start_offset" ] * time_offset, 2 )
    end_offset = round (w[ "end_offset" ] * time_offset, 2 )
    print('word : ',w['char'],'start_offset :',start_offset,'end_offset :',end_offset)
    # display(audio_input[start_offset*1000:end_offset*1000])

Channels: 1
Sample rate: 16000
Duration:  1.637
Bit depth: 2 bits
len samples: 26192


word :  ㅇ start_offset : 0.0 end_offset : 0.02
word :  ㅡ start_offset : 0.02 end_offset : 0.04
word :  ㅁ start_offset : 0.06 end_offset : 0.08
word :  ㅅ start_offset : 0.28 end_offset : 0.32
word :  ㅓ start_offset : 0.32 end_offset : 0.34
word :  ㅇ start_offset : 0.42 end_offset : 0.44
word :    start_offset : 0.5 end_offset : 0.52
word :  ㅇ start_offset : 0.6 end_offset : 0.62
word :  ㅣ start_offset : 0.62 end_offset : 0.64
word :  ㄴ start_offset : 0.68 end_offset : 0.7
word :  ㅅ start_offset : 0.86 end_offset : 0.88
word :  ㅣ start_offset : 0.9 end_offset : 0.92
word :  ㄱ start_offset : 0.96 end_offset : 1.0
word :  ㅌ start_offset : 1.04 end_offset : 1.06
word :  ㅔ start_offset : 1.06 end_offset : 1.08
word :  ㅅ start_offset : 1.28 end_offset : 1.3
word :  ㅡ start_offset : 1.3 end_offset : 1.32
word :  ㅌ start_offset : 1.48 end_offset : 1.5
word :  ㅡ start_offset : 1.52 end_offset : 1.54
